# Introduction to Notebook 
This notebook shows a handson session of the application of Topic Modelling using Gensim, a popular python library. The notebook will go through how to import an API based dataset, how to apply Latent Semantic Analysis (LSA) and Latent Dirichlet Allocation (LDA)​ algorithm and how to measure the accuracy of the topic modelling model using the coherence score. 



# Dataset
The dataset that will be used as a sample in this notebook is the [Twenty Newsgroups Data](https://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups) Set from the open source UCI Machine Learning Repository. For the purpose of this exercise, a cleaned version of this dataset will be imported from the [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html?highlight=newsgroup#sklearn.datasets.fetch_20newsgroups) API

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import pandas as pd

In [ ]:
dataset = fetch_20newsgroups(subset = 'all',shuffle= False, random_state=32,remove=('headers', 'footers', 'qutes'))

In [ ]:
dataset.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
dataset['data']

In [ ]:
pprint(dataset.target_names)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [ ]:
len(dataset.target_names)

20

In [ ]:
dataset.target.shape

(18846,)

In [ ]:
dataset.filenames.shape

(18846,)

In [ ]:
dataset_df = pd.DataFrame({'News':dataset.data, 'Label' : dataset.target})

In [ ]:
dataset_df

News  Label
0      gajarsky@pilot.njin.net writes:\n\nmorgan and ...      9
1      Well, I just got my Centris 610 yesterday.  It...      4
2      Archive-name: cryptography-faq/part10\nLast-mo...     11
3      > ATTENTION: Mac Quadra owners: Many storage i...      4
4      bobbe@vice.ICO.TEK.COM (Robert Beauchaine) wri...      0
...                                                  ...    ...
18841  \nWhy are circuit boards green?  The material ...     12
18842  In article <1r941o$3tu@menudo.uh.edu> inde7wv@...      8
18843  We were told that the resolution on the 5FGe c...      4
18844  CAD Setup For Sale:\n\nG486PLB Local Bus Mothe...      6
18845  In article <1r8uckINNcmf@gap.caltech.edu> wen-...      7

[18846 rows x 2 columns]

In [ ]:
dataset_df['Label_name'] = dataset_df['Label'].apply(lambda x: dataset.target_names[x])

In [ ]:
dataset_df

News  Label  \
0      gajarsky@pilot.njin.net writes:\n\nmorgan and ...      9   
1      Well, I just got my Centris 610 yesterday.  It...      4   
2      Archive-name: cryptography-faq/part10\nLast-mo...     11   
3      > ATTENTION: Mac Quadra owners: Many storage i...      4   
4      bobbe@vice.ICO.TEK.COM (Robert Beauchaine) wri...      0   
...                                                  ...    ...   
18841  \nWhy are circuit boards green?  The material ...     12   
18842  In article <1r941o$3tu@menudo.uh.edu> inde7wv@...      8   
18843  We were told that the resolution on the 5FGe c...      4   
18844  CAD Setup For Sale:\n\nG486PLB Local Bus Mothe...      6   
18845  In article <1r8uckINNcmf@gap.caltech.edu> wen-...      7   

                  Label_name  
0         rec.sport.baseball  
1      comp.sys.mac.hardware  
2                  sci.crypt  
3      comp.sys.mac.hardware  
4                alt.atheism  
...                      ...  
18841        sci.electronics  
18842        rec.motorcycles  
18843  comp.sys.mac.hardware  
18844           misc.forsale  
18845              rec.autos  

[18846 rows x 3 columns]

# Dataset preprocessing

In [ ]:
%%capture
!pip install -U gensim

In [ ]:
from gensim.utils import tokenize
from gensim.parsing.preprocessing import preprocess_string,strip_tags,strip_punctuation,strip_numeric,remove_stopwords,strip_short
from gensim.corpora.dictionary import Dictionary
from gensim import models

In [ ]:
help(preprocess_string)

Help on function preprocess_string in module gensim.parsing.preprocessing:

preprocess_string(s, filters=[<function <lambda> at 0x7f167ffe9950>, <function strip_tags at 0x7f167ffe1050>, <function strip_punctuation at 0x7f167ffdca70>, <function strip_multiple_whitespaces at 0x7f167ffe1ef0>, <function strip_numeric at 0x7f167ffe1a70>, <function remove_stopwords at 0x7f167ff3c0e0>, <function strip_short at 0x7f167ffe1680>, <function stem_text at 0x7f167ffe98c0>])
    Apply list of chosen filters to `s`.
    
    Default list of filters:
    
    * :func:`~gensim.parsing.preprocessing.strip_tags`,
    * :func:`~gensim.parsing.preprocessing.strip_punctuation`,
    * :func:`~gensim.parsing.preprocessing.strip_multiple_whitespaces`,
    * :func:`~gensim.parsing.preprocessing.strip_numeric`,
    * :func:`~gensim.parsing.preprocessing.remove_stopwords`,
    * :func:`~gensim.parsing.preprocessing.strip_short`,
    * :func:`~gensim.parsing.preprocessing.stem_text`.
    
    Parameters
    -------

In [ ]:
dataset_df['Clean_news'] = dataset_df['News'].apply(preprocess_string)

In [ ]:
dataset_df

News  Label  \
0      gajarsky@pilot.njin.net writes:\n\nmorgan and ...      9   
1      Well, I just got my Centris 610 yesterday.  It...      4   
2      Archive-name: cryptography-faq/part10\nLast-mo...     11   
3      > ATTENTION: Mac Quadra owners: Many storage i...      4   
4      bobbe@vice.ICO.TEK.COM (Robert Beauchaine) wri...      0   
...                                                  ...    ...   
18841  \nWhy are circuit boards green?  The material ...     12   
18842  In article <1r941o$3tu@menudo.uh.edu> inde7wv@...      8   
18843  We were told that the resolution on the 5FGe c...      4   
18844  CAD Setup For Sale:\n\nG486PLB Local Bus Mothe...      6   
18845  In article <1r8uckINNcmf@gap.caltech.edu> wen-...      7   

                  Label_name  \
0         rec.sport.baseball   
1      comp.sys.mac.hardware   
2                  sci.crypt   
3      comp.sys.mac.hardware   
4                alt.atheism   
...                      ...   
18841        sci.electronics   
18842        rec.motorcycles   
18843  comp.sys.mac.hardware   
18844           misc.forsale   
18845              rec.autos   

                                              Clean_news  
0      [gajarski, pilot, njin, net, write, morgan, gu...  
1      [got, centri, yesterdai, took, week, place, or...  
2      [archiv, cryptographi, faq, modifi, faq, sci, ...  
3      [attent, mac, quadra, owner, storag, industri,...  
4      [bobb, vice, ico, tek, com, robert, beauchain,...  
...                                                  ...  
18841  [circuit, board, green, materi, goe, name, cir...  
18842  [articl, indewv, rosi, edu, write, bike, lucki...  
18843  [told, resolut, fge, anybodi, tri, run, higher...  
18844  [cad, setup, sale, gplb, local, bu, motherboar...  
18845  [articl, wen, king, caltech, edu, wen, king, w...  

[18846 rows x 4 columns]

In [ ]:
filters=[lambda x: x.lower(),strip_tags,strip_punctuation,strip_numeric,remove_stopwords,strip_short]
dataset_df['Clean_news1'] = dataset_df['News'].apply(lambda x: preprocess_string(x,filters))

In [ ]:
dataset_df

News  Label  \
0      gajarsky@pilot.njin.net writes:\n\nmorgan and ...      9   
1      Well, I just got my Centris 610 yesterday.  It...      4   
2      Archive-name: cryptography-faq/part10\nLast-mo...     11   
3      > ATTENTION: Mac Quadra owners: Many storage i...      4   
4      bobbe@vice.ICO.TEK.COM (Robert Beauchaine) wri...      0   
...                                                  ...    ...   
18841  \nWhy are circuit boards green?  The material ...     12   
18842  In article <1r941o$3tu@menudo.uh.edu> inde7wv@...      8   
18843  We were told that the resolution on the 5FGe c...      4   
18844  CAD Setup For Sale:\n\nG486PLB Local Bus Mothe...      6   
18845  In article <1r8uckINNcmf@gap.caltech.edu> wen-...      7   

                  Label_name  \
0         rec.sport.baseball   
1      comp.sys.mac.hardware   
2                  sci.crypt   
3      comp.sys.mac.hardware   
4                alt.atheism   
...                      ...   
18841        sci.electronics   
18842        rec.motorcycles   
18843  comp.sys.mac.hardware   
18844           misc.forsale   
18845              rec.autos   

                                              Clean_news  \
0      [gajarski, pilot, njin, net, write, morgan, gu...   
1      [got, centri, yesterdai, took, week, place, or...   
2      [archiv, cryptographi, faq, modifi, faq, sci, ...   
3      [attent, mac, quadra, owner, storag, industri,...   
4      [bobb, vice, ico, tek, com, robert, beauchain,...   
...                                                  ...   
18841  [circuit, board, green, materi, goe, name, cir...   
18842  [articl, indewv, rosi, edu, write, bike, lucki...   
18843  [told, resolut, fge, anybodi, tri, run, higher...   
18844  [cad, setup, sale, gplb, local, bu, motherboar...   
18845  [articl, wen, king, caltech, edu, wen, king, w...   

                                             Clean_news1  
0      [gajarsky, pilot, njin, net, writes, morgan, g...  
1      [got, centris, yesterday, took, weeks, placing...  
2      [archive, cryptography, faq, modified, faq, sc...  
3      [attention, mac, quadra, owners, storage, indu...  
4      [bobbe, vice, ico, tek, com, robert, beauchain...  
...                                                  ...  
18841  [circuit, boards, green, material, goes, names...  
18842  [article, indewv, rosie, edu, writes, bike, lu...  
18843  [told, resolution, fge, anybody, tried, runnin...  
18844  [cad, setup, sale, gplb, local, bus, motherboa...  
18845  [article, wen, king, caltech, edu, wen, king, ...  

[18846 rows x 5 columns]

In [ ]:
dataset_dictionary = Dictionary(dataset_df['Clean_news1'])

In [ ]:
len(dataset_dictionary)

96460

In [ ]:
print(dataset_dictionary.token2id)

{'castillo': 0, 'cubs': 1, 'era': 2, 'gajarsky': 3, 'good': 4, 'guzman': 5, 'harkey': 6, 'hibbard': 7, 'higher': 8, 'idiots': 9, 'morgan': 10, 'net': 11, 'njin': 12, 'pilot': 13, 'pitch': 14, 'pitcher': 15, 'run': 16, 'stud': 17, 'think': 18, 'won': 19, 'writes': 20, 'year': 21, 'alright': 22, 'annoying': 23, 'apple': 24, 'appologized': 25, 'awful': 26, 'backlogged': 27, 'big': 28, 'boot': 29, 'buttons': 30, 'cable': 31, 'centri': 32, 'centris': 33, 'come': 34, 'complain': 35, 'connected': 36, 'control': 37, 'convenient': 38, 'deal': 39, 'dealer': 40, 'decided': 41, 'dissappionted': 42, 'drive': 43, 'ethernet': 44, 'exceptions': 45, 'exchange': 46, 'fast': 47, 'feature': 48, 'functions': 49, 'got': 50, 'great': 51, 'happy': 52, 'hard': 53, 'having': 54, 'heard': 55, 'high': 56, 'interupt': 57, 'keyboard': 58, 'know': 59, 'lack': 60, 'literature': 61, 'location': 62, 'loose': 63, 'machine': 64, 'mentioned': 65, 'nosy': 66, 'open': 67, 'order': 68, 'ordered': 69, 'partially': 70, 'pissed

In [ ]:
dataset_corpus_bow = [dataset_dictionary.doc2bow(text) for text in dataset_df['Clean_news1']] #create a dataset corpus with bag of word vectorization

In [ ]:
len(dataset_corpus_bow)

18846

In [ ]:
print(dataset_corpus_bow[1])

[(22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 3), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 2), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1)]


In [ ]:
tfidf = models.TfidfModel(dataset_corpus_bow)
dataset_corpus_tfidf = tfidf[dataset_corpus_bow]

In [ ]:
len(dataset_corpus_tfidf)

18846

In [ ]:
print(dataset_corpus_tfidf[1])

[(22, 0.12794312043780054), (23, 0.1032933602529823), (24, 0.1437906445046912), (25, 0.19446130648981633), (26, 0.09972437101248886), (27, 0.19446130648981633), (28, 0.056593976938038), (29, 0.09712742378308543), (30, 0.11391287593244794), (31, 0.08578843198010519), (32, 0.19446130648981633), (33, 0.10142837384435295), (34, 0.09687548915310225), (35, 0.10125120507490903), (36, 0.09133742977605598), (37, 0.061307837508357034), (38, 0.10891119725810347), (39, 0.06606330836365701), (40, 0.08855334717656915), (41, 0.07391023272465086), (42, 0.19446130648981633), (43, 0.11381832135728932), (44, 0.3110653713924378), (45, 0.11566217615948503), (46, 0.0969847936661337), (47, 0.0694228263237942), (48, 0.09383842074796034), (49, 0.09300779209433964), (50, 0.0455655162646588), (51, 0.10391838009790301), (52, 0.07696272067182856), (53, 0.10763740852872361), (54, 0.05222277834588352), (55, 0.05433687654103351), (56, 0.05404373403325169), (57, 0.15338363279035241), (58, 0.08979736303341844), (59, 0.

# Topic Modelling with Latent Dirichlet Allocation(LDA)

In [ ]:
from gensim.models.ldamodel import LdaModel

In [ ]:
lda_bow = LdaModel(dataset_corpus_bow,num_topics=20,id2word=dataset_dictionary,random_state=0)

In [ ]:
lda_topics_bow = lda_bow.print_topics(num_words=8)
for topic in lda_topics_bow:
  print(topic)

(0, '0.013*"space" + 0.006*"nasa" + 0.006*"earth" + 0.006*"writes" + 0.005*"edu" + 0.005*"shuttle" + 0.004*"gamma" + 0.004*"article"')
(1, '0.014*"edu" + 0.008*"university" + 0.008*"mail" + 0.007*"information" + 0.006*"list" + 0.005*"com" + 0.005*"address" + 0.005*"research"')
(2, '0.014*"writes" + 0.012*"article" + 0.010*"like" + 0.007*"car" + 0.007*"edu" + 0.007*"people" + 0.006*"think" + 0.006*"know"')
(3, '0.007*"health" + 0.006*"edu" + 0.005*"writes" + 0.005*"insurance" + 0.005*"private" + 0.004*"pictures" + 0.004*"canada" + 0.003*"care"')
(4, '0.006*"said" + 0.006*"people" + 0.006*"know" + 0.006*"time" + 0.005*"like" + 0.005*"think" + 0.004*"children" + 0.004*"writes"')
(5, '0.008*"use" + 0.008*"drive" + 0.007*"know" + 0.007*"bit" + 0.006*"like" + 0.006*"problem" + 0.006*"mac" + 0.005*"writes"')
(6, '0.011*"article" + 0.011*"writes" + 0.010*"people" + 0.007*"evidence" + 0.007*"edu" + 0.006*"think" + 0.005*"law" + 0.004*"homosexual"')
(7, '0.010*"key" + 0.007*"government" + 0.007*

In [ ]:
lda_tfidf = LdaModel(dataset_corpus_tfidf, id2word=dataset_dictionary, num_topics=20)

In [ ]:
lda_topics_tfidf = lda_tfidf.print_topics(num_words=8)
for topic in lda_topics_tfidf:
  print(topic)

(0, '0.003*"people" + 0.002*"article" + 0.002*"god" + 0.002*"think" + 0.002*"writes" + 0.002*"edu" + 0.002*"right" + 0.002*"like"')
(1, '0.002*"gant" + 0.002*"gay" + 0.002*"hitter" + 0.002*"boston" + 0.001*"associate" + 0.001*"mattingly" + 0.001*"surgery" + 0.001*"proving"')
(2, '0.005*"monitor" + 0.003*"video" + 0.003*"vga" + 0.002*"baden" + 0.002*"eisa" + 0.002*"cica" + 0.002*"cancer" + 0.002*"ati"')
(3, '0.003*"images" + 0.003*"hst" + 0.002*"solar" + 0.002*"moon" + 0.002*"shareware" + 0.002*"jpeg" + 0.002*"batteries" + 0.002*"iici"')
(4, '0.002*"subscribe" + 0.001*"bil" + 0.001*"okcforum" + 0.001*"osrhe" + 0.001*"kinsey" + 0.001*"conner" + 0.001*"min" + 0.001*"harddisk"')
(5, '0.004*"windows" + 0.003*"thanks" + 0.003*"drive" + 0.003*"program" + 0.002*"file" + 0.002*"dos" + 0.002*"mail" + 0.002*"know"')
(6, '0.002*"fuhr" + 0.002*"maynard" + 0.001*"bruins" + 0.001*"luke" + 0.001*"infection" + 0.001*"courier" + 0.001*"boeing" + 0.001*"laserwriter"')
(7, '0.003*"modems" + 0.002*"bosnian

# Topic Modelling with Latent Semantic Analysis/Indexing(LSA/LSI)

In [ ]:
from gensim.models.lsimodel import LsiModel

In [ ]:
lsi_bow = LsiModel(corpus=dataset_corpus_bow,id2word=dataset_dictionary,num_topics=20)

In [ ]:
lsi_topics_bow = lsi_bow.print_topics(num_words=8)
for topic in lsi_topics_bow:
  print(topic)

(0, '0.994*"max" + 0.069*"giz" + 0.068*"bhj" + 0.025*"qax" + 0.015*"biz" + 0.014*"nrhj" + 0.014*"bxn" + 0.012*"nuy"')
(1, '0.255*"jpeg" + 0.253*"file" + 0.219*"edu" + 0.204*"image" + 0.204*"dos" + 0.164*"use" + 0.153*"available" + 0.137*"ftp"')
(2, '-0.772*"dos" + -0.277*"windows" + -0.147*"microsoft" + 0.113*"people" + -0.109*"tcp" + 0.090*"jpeg" + -0.090*"mouse" + 0.083*"know"')
(3, '-0.435*"jpeg" + -0.231*"image" + 0.216*"people" + 0.174*"said" + 0.171*"dos" + -0.167*"file" + -0.166*"gif" + 0.166*"know"')
(4, '-0.442*"jpeg" + 0.297*"edu" + -0.190*"dos" + 0.171*"pub" + -0.158*"gif" + -0.140*"people" + 0.138*"com" + 0.138*"ftp"')
(5, '0.439*"god" + 0.366*"jehovah" + 0.297*"lord" + 0.275*"elohim" + -0.200*"file" + 0.183*"christ" + 0.177*"jesus" + 0.141*"father"')
(6, '-0.640*"file" + -0.204*"gun" + 0.164*"edu" + 0.162*"image" + -0.134*"jehovah" + -0.132*"god" + -0.115*"output" + -0.110*"control"')
(7, '0.350*"stephanopoulos" + 0.267*"president" + -0.221*"use" + 0.221*"file" + 0.190*"gr

In [ ]:
lsi_tfidf = LsiModel(dataset_corpus_tfidf, id2word=dataset_dictionary, num_topics=20)

In [ ]:
lsi_topics_tfidf = lsi_tfidf.print_topics(num_words=8)
for topic in lsi_topics_tfidf:
  print(topic)

(0, '0.127*"people" + 0.122*"god" + 0.110*"know" + 0.109*"like" + 0.101*"think" + 0.099*"windows" + 0.094*"edu" + 0.093*"use"')
(1, '0.279*"god" + -0.243*"windows" + -0.194*"drive" + -0.170*"card" + -0.164*"scsi" + -0.153*"dos" + -0.125*"thanks" + 0.123*"jesus"')
(2, '0.480*"god" + 0.199*"jesus" + -0.191*"game" + -0.136*"team" + 0.126*"bible" + 0.124*"windows" + 0.123*"scsi" + -0.121*"games"')
(3, '0.259*"game" + -0.241*"key" + 0.214*"drive" + 0.192*"scsi" + -0.176*"encryption" + -0.168*"clipper" + -0.167*"chip" + 0.162*"team"')
(4, '0.381*"scsi" + 0.367*"drive" + -0.316*"windows" + 0.196*"ide" + -0.147*"file" + 0.137*"controller" + 0.126*"chip" + 0.126*"drives"')
(5, '0.277*"key" + 0.276*"god" + 0.217*"chip" + -0.205*"israel" + 0.202*"game" + 0.174*"clipper" + 0.167*"encryption" + 0.138*"keys"')
(6, '-0.307*"car" + 0.252*"windows" + 0.213*"israel" + 0.206*"scsi" + 0.187*"dos" + 0.140*"game" + -0.132*"bike" + 0.126*"israeli"')
(7, '-0.444*"card" + 0.268*"drive" + -0.223*"video" + -0.20